# Navigation

---

In this notebook, you will learn how to use the Unity ML-Agents environment for the first project of the [Deep Reinforcement Learning Nanodegree](https://www.udacity.com/course/deep-reinforcement-learning-nanodegree--nd893).

### 1. Start the Environment

We begin by importing some necessary packages.  If the code cell below returns an error, please revisit the project instructions to double-check that you have installed [Unity ML-Agents](https://github.com/Unity-Technologies/ml-agents/blob/master/docs/Installation.md) and [NumPy](http://www.numpy.org/).

In [ ]:
from unityagents import UnityEnvironment
import numpy as np
import torch.nn as nn
import torch.nn.functional as F

Next, we will start the environment!  **_Before running the code cell below_**, change the `file_name` parameter to match the location of the Unity environment that you downloaded.

- **Mac**: `"path/to/Banana.app"`
- **Windows** (x86): `"path/to/Banana_Windows_x86/Banana.exe"`
- **Windows** (x86_64): `"path/to/Banana_Windows_x86_64/Banana.exe"`
- **Linux** (x86): `"path/to/Banana_Linux/Banana.x86"`
- **Linux** (x86_64): `"path/to/Banana_Linux/Banana.x86_64"`
- **Linux** (x86, headless): `"path/to/Banana_Linux_NoVis/Banana.x86"`
- **Linux** (x86_64, headless): `"path/to/Banana_Linux_NoVis/Banana.x86_64"`

For instance, if you are using a Mac, then you downloaded `Banana.app`.  If this file is in the same folder as the notebook, then the line below should appear as follows:
```
env = UnityEnvironment(file_name="Banana.app")
```

In [ ]:
env = UnityEnvironment(file_name="Banana_Windows_x86_64/Banana.exe")

Environments contain **_brains_** which are responsible for deciding the actions of their associated agents. Here we check for the first brain available, and set it as the default brain we will be controlling from Python.

In [ ]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

### 2. Examine the State and Action Spaces

The simulation contains a single agent that navigates a large environment.  At each time step, it has four actions at its disposal:
- `0` - walk forward 
- `1` - walk backward
- `2` - turn left
- `3` - turn right

The state space has `37` dimensions and contains the agent's velocity, along with ray-based perception of objects around agent's forward direction.  A reward of `+1` is provided for collecting a yellow banana, and a reward of `-1` is provided for collecting a blue banana. 

Run the code cell below to print some information about the environment.

In [ ]:
# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents in the environment
print('Number of agents:', len(env_info.agents))

# number of actions
action_size = brain.vector_action_space_size
print('Number of actions:', action_size)

# examine the state space 
state = env_info.vector_observations[0]
print('States look like:', state)
state_size = len(state)
print('States have length:', state_size)
action = np.random.randint(action_size) 
print(type(action))
if type(action) is not int:
    print(1222)

### 3. Take Random Actions in the Environment

In the next code cell, you will learn how to use the Python API to control the agent and receive feedback from the environment.

Once this cell is executed, you will watch the agent's performance, if it selects an action (uniformly) at random with each time step.  A window should pop up that allows you to observe the agent, as it moves through the environment.  

Of course, as part of the project, you'll have to change the code so that the agent is able to use its experience to gradually choose better actions when interacting with the environment!

In [ ]:
env_info = env.reset(train_mode=False)[brain_name] # reset the environment
state = env_info.vector_observations[0]            # get the current state
score = 0                                          # initialize the score
while True:
    action = np.random.randint(action_size)        # select an action
    env_info = env.step(action)[brain_name]        # send the action to the environment
    next_state = env_info.vector_observations[0]   # get the next state
    reward = env_info.rewards[0]                   # get the reward
    done = env_info.local_done[0]                  # see if episode has finished
    score += reward                                # update the score
    state = next_state                             # roll over the state to next time step
    if done:                                       # exit loop if episode finished
        break
    
print("Score: {}".format(score))

In [ ]:
env_info = env.reset(train_mode=True)[brain_name] # reset the environment
state = env_info.vector_observations[0]            # get the current state
score = 0                                          # initialize the score
while True:
    action = np.random.randint(action_size)        # select an action
    env_info = env.step(action)[brain_name]        # send the action to the environment
    next_state = env_info.vector_observations[0]   # get the next state
    reward = env_info.rewards[0]                   # get the reward
    done = env_info.local_done[0]                  # see if episode has finished
    score += reward                                # update the score
    state = next_state                             # roll over the state to next time step
    if done:                                       # exit loop if episode finished
        break
    
print("Score: {}".format(score))

### ==================================
### Run the code cell below to train an agent
### ==================================

In [1]:
from unityagents import UnityEnvironment
from model import QNetwork
from dqn_agent import Agent
import random
import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
from collections import namedtuple, deque


env = UnityEnvironment(file_name="Banana_Windows_x86_64/Banana.exe")
brain_name = env.brain_names[0]
brain = env.brains[brain_name]
env_info = env.reset(train_mode=True)[brain_name]
action_size = brain.vector_action_space_size
state = env_info.vector_observations[0]
state_size = len(state)
num_episodes = 2000
alpha = 0.0005
gamma = 0.99
tau = 0.001
agent = Agent(state_size, action_size, alpha, gamma, tau)

def Deep_Q_Learning(env, agent, num_episodes, alpha, gamma, epsilon=1.0, epsilon_decay=0.992, epsilon_min=0.01):
    scores_window = deque(maxlen=100)
    scores = []
    # Loop through the specified episodes
    for i_episode in range(1, num_episodes+1):
        score = 0
        # Reset the environment 
        env_info = env.reset(train_mode=True)[brain_name]
        state = env_info.vector_observations[0]
        
        while True:
            # Get the first action of the agent and convert it to standard python type int
            action = agent.act(state, epsilon)
            if type(action) is not int:
                action = action.item()
            # Take a step with the action in the environment and get the next state and reward         
            env_info = env.step(action)[brain_name]
            next_state = env_info.vector_observations[0]
            reward = env_info.rewards[0]
            agent.step(state, action, reward, next_state)
            state = next_state
            score += reward 
            done = env_info.local_done[0]
            # exit loop if episode finished
            if done:
                break
                    
        # Calculate epsilon for the epsilon greedy policy to not always use the greedy action
        # A highger value means to take random actions, a lower value means to take the action 
        # which gives the highest expected reward
        epsilon = max(epsilon*epsilon_decay, epsilon_min) 
        
        scores_window.append(score)  # save most recent score
        scores.append(score)  # save most recent score
        if i_episode % 20 == 0:
            print('\rEpisode {}\tAverage Score: {:.2f}'.format(i_episode, np.mean(scores_window)))

        if np.mean(scores_window) >= 13.0:
            print('\nEnvironment solved in {:d} episodes!\tAverage Score: {:.2f}'.format(i_episode - 100,
                                                                                         np.mean(scores_window)))
            checkpoint = {'input_size': state_size,
                          'output_size': action_size,
                          'hidden_layers': [each.out_features for each in agent.model_local.hidden_layers],
                          'state_dict': agent.model_local.state_dict()}
            torch.save(checkpoint, 'checkpoint.pth')
            break
    return scores


# Start Deep Q Learning
Deep_Q_Learning(env, agent, num_episodes, alpha, gamma)

        
    

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		
Unity brain name: BananaBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 37
        Number of stacked Vector Observation: 1
        Vector Action space type: discrete
        Vector Action space size (per agent): 4
        Vector Action descriptions: , , , 


Episode 20	Average Score: -0.25
Episode 40	Average Score: 0.17
Episode 60	Average Score: 0.22
Episode 80	Average Score: 0.34
Episode 100	Average Score: 0.49
Episode 120	Average Score: 1.10
Episode 140	Average Score: 1.33
Episode 160	Average Score: 2.03
Episode 180	Average Score: 2.95
Episode 200	Average Score: 3.97
Episode 220	Average Score: 4.59
Episode 240	Average Score: 5.31
Episode 260	Average Score: 5.78
Episode 280	Average Score: 6.01
Episode 300	Average Score: 6.22
Episode 320	Average Score: 6.62
Episode 340	Average Score: 7.14
Episode 360	Average Score: 7.28
Episode 380	Average Score: 7.61
Episode 400	Average Score: 8.29
Episode 420	Average Score: 9.09
Episode 440	Average Score: 10.10
Episode 460	Average Score: 11.05
Episode 480	Average Score: 12.05
Episode 500	Average Score: 12.24
Episode 520	Average Score: 12.91
Episode 540	Average Score: 12.54

Environment solved in 452 episodes!	Average Score: 13.07


[0.0,
 1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 0.0,
 0.0,
 1.0,
 0.0,
 -1.0,
 0.0,
 -2.0,
 0.0,
 1.0,
 0.0,
 0.0,
 1.0,
 -1.0,
 -1.0,
 0.0,
 -1.0,
 -1.0,
 0.0,
 0.0,
 0.0,
 1.0,
 3.0,
 2.0,
 1.0,
 1.0,
 3.0,
 0.0,
 -1.0,
 1.0,
 2.0,
 -2.0,
 0.0,
 3.0,
 0.0,
 0.0,
 5.0,
 -1.0,
 2.0,
 2.0,
 4.0,
 0.0,
 -1.0,
 0.0,
 -1.0,
 0.0,
 -1.0,
 -2.0,
 0.0,
 -2.0,
 1.0,
 0.0,
 1.0,
 -2.0,
 1.0,
 1.0,
 1.0,
 2.0,
 1.0,
 1.0,
 -3.0,
 3.0,
 0.0,
 1.0,
 1.0,
 1.0,
 0.0,
 0.0,
 -2.0,
 2.0,
 -1.0,
 3.0,
 -1.0,
 2.0,
 2.0,
 2.0,
 3.0,
 1.0,
 1.0,
 -1.0,
 3.0,
 0.0,
 2.0,
 3.0,
 1.0,
 -1.0,
 1.0,
 0.0,
 2.0,
 3.0,
 2.0,
 2.0,
 1.0,
 -2.0,
 -1.0,
 1.0,
 6.0,
 5.0,
 4.0,
 4.0,
 5.0,
 3.0,
 0.0,
 2.0,
 2.0,
 4.0,
 3.0,
 1.0,
 1.0,
 1.0,
 2.0,
 2.0,
 2.0,
 6.0,
 2.0,
 3.0,
 1.0,
 0.0,
 0.0,
 1.0,
 1.0,
 2.0,
 5.0,
 0.0,
 0.0,
 3.0,
 5.0,
 2.0,
 2.0,
 2.0,
 -2.0,
 1.0,
 1.0,
 5.0,
 3.0,
 3.0,
 7.0,
 2.0,
 6.0,
 5.0,
 1.0,
 3.0,
 7.0,
 9.0,
 6.0,
 2.0,
 -2.0,
 3.0,
 3.0,
 2.0,
 1.0,
 7.0,
 3.0,
 4.0,
 4.0,
 1.0,
 5.0,

When finished, you can close the environment.

### =====================================================
### Run the code cell below to load an agent and see how it behaves
### =====================================================

In [ ]:
from unityagents import UnityEnvironment
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from model import QNetwork

def load_checkpoint(filepath):
    checkpoint = torch.load(filepath)
    model = QNetwork(checkpoint['input_size'],
                        checkpoint['output_size'],
                        checkpoint['hidden_layers'])
    model.load_state_dict(checkpoint['state_dict'])
    
    return model

env = UnityEnvironment(file_name="Banana_Windows_x86_64/Banana.exe")
brain_name = env.brain_names[0]
brain = env.brains[brain_name]
model = load_checkpoint('checkpoint.pth')
env_info = env.reset(train_mode=False)[brain_name] # reset the environment
state = env_info.vector_observations[0]            # get the current state
score = 0                                          # initialize the score
num_episode = 10
for i in range(num_episode):
    score = 0
    env_info = env.reset(train_mode=False)[brain_name]
    state = env_info.vector_observations[0]
    while True:
        state_t = torch.from_numpy(state).float().unsqueeze(0)
        action_value = model(state_t)
        action = np.argmax(action_value.data.numpy()) 
        if type(action) is not int:
                    action = action.item()
        env_info = env.step(action)[brain_name]        # send the action to the environment
        next_state = env_info.vector_observations[0]   # get the next state
        reward = env_info.rewards[0]
        score += reward                                # update the score
        state = next_state                             # roll over the state to next time step
        done = env_info.local_done[0]
        if done:                                       # exit loop if episode finished
            break
    
    print("Score: {}".format(score))



### 4. It's Your Turn!

Now it's your turn to train your own agent to solve the environment!  When training the environment, set `train_mode=True`, so that the line for resetting the environment looks like the following:
```python
env_info = env.reset(train_mode=True)[brain_name]
```